### The following setup draws from the Mask_RCNN repo by matterport and Deep Learning with Python by Chollet. 

https://github.com/matterport/Mask_RCNN
https://github.com/fchollet/deep-learning-with-python-notebooks

We import our packages, including maskrcnn, which needs to be installed from the github repo. 

We also set up our directories and paths before we organize our data into tensors. 

We subclass the dataset and config classes for our specific dataset

Then, we train the model and test.

TO DO:
- Try to prepare the dataset and see if Keras trains succesfully, with loss decreasing at each step.

- Try data augmentation: image rotation and flipping to increase our training set 6 fold

- Explore data aug options in load_image_gt():  

        augmentation: Optional. An imgaug (https://github.com/aleju/imgaug) augmentation.
        For example, passing imgaug.augmenters.Fliplr(0.5) flips images
        right/left 50% of the time.

- change Config attributes to see if hyperparameters like anchor sizes (size of proposed regions that objects are located in) dramatically impact model training time and performance


In [1]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt


# Import Mask RCNN
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.config import Config
from mrcnn.model import log

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "models")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(MODEL_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)
EIGHTCHANNELDIR = os.path.join(ROOT_DIR, 'data/raw/eightchannels')
TRAIN_DIR = os.path.join(ROOT_DIR, 'data/raw/train')
VALIDATION_DIR = os.path.join(ROOT_DIR, 'data/raw/validation')
TEST_DIR = os.path.join(ROOT_DIR, 'data/raw/test')
try:
    os.mkdir(train_dir)
    os.mkdir(validation_dir)
    os.mkdir(test_dir)
except:
    FileExistsError

/opt/anaconda3/envs/maskrcnn/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
### Should our tensor be rank 4 [h, w, channels on and off season, instance] 
### or rank 5 [h, w, channels on season, instances, channels off season]
def load_merge_wv2(image_id):
    """Load the specified wv2 os/gs image pairs and return a [H,W,8] 
    Numpy array. Channels are ordered [B, G, R, NIR, B, G, R, NIR], OS 
    first.
    """
    # Load image
    os_path = IMAGERY_DIR+'/'+image_id+'_OS_ms.tif'
    gs_path = IMAGERY_DIR+'/'+image_id+'_GS_ms.tif'
    os_image = skimage.io.imread(os_path)
    gs_image = skimage.io.imread(gs_path)
    # If has more than 4 bands, select correct bands 
    # will need to provide image config in future
    # to programmaticaly use correct band mappings
    if os_image.shape[-1] != 4:
        os_image = np.dstack((os_image[:,:,1:3],os_image[:,:,4],os_image[:,:,6]))
    if gs_image.shape[-1] != 4:
        gs_image = np.dstack((gs_image[:,:,1:3],gs_image[:,:,4],gs_image[:,:,6]))
    stacked_image = np.dstack((os_image, gs_image))
    stacked_image_path = EIGHTCHANNELDIR +'/'+ image_id + '_OSGS_ms.tif'
    return (stacked_image_path, stacked_image)

IMAGERY_DIR = os.path.join(ROOT_DIR, 'data/raw/stephtest-subset/Imagery')
GROUNDTRUTH_DIR = os.path.join(ROOT_DIR, 'data/raw/stephtest-subset/Groundtruth')

# all files, including ones we don't care about
file_ids_all = next(os.walk(IMAGERY_DIR))[2]
# all multispectral on and off season tifs
image_ids_all = [image_id for image_id in file_ids_all if 'ms' in image_id]
#check for duplicates
print(len(image_ids_all) != len(set(image_ids_all)))

image_ids_gs = [image_id for image_id in image_ids_all if 'GS' in image_id]
image_ids_os = [image_id for image_id in image_ids_all if 'OS' in image_id]

#check for equality
print(len(image_ids_os) == len(image_ids_gs))

image_ids_short = [image_id[0:8] for image_id in image_ids_gs]
image_ids_short

stacked_dict = {}

for imid in image_ids_short:
    
    path, arr = load_merge_wv2(imid)
    stacked_dict.update({path:arr})
    
# trying to save 8 channel numpy array with GS, OS info. this is what matterport expects
# BUT in mold_inputs() in https://github.com/matterport/Mask_RCNN/blob/master/mrcnn/model.py
# indicates that the input png/array must only have three channels...
# We could change mold_inputs to not have this requirement and also change
#         input_image = KL.Input(
#            shape=[None, None, 3], name="input_image")
# on line 1841 of mrcnn/model.py but not sure if this is all the changes
# that would be required
# this issue indicates fix is simpler: https://github.com/matterport/Mask_RCNN/issues/314

for key, val in stacked_dict.items():
    skimage.io.imsave(key,val,plugin='tifffile')

False
True


In [41]:
import random
import shutil
random.seed(42)

def train_test_split(imagerydir, traindir, testdir, kprop):
    """Splits tifs into train and test dir."""
    
    image_list = next(os.walk(imagerydir))[2]
    k = round(kprop*len(image_list))
    test_list = random.sample(image_list,k)
    for test in test_list:
        shutil.copyfile(os.path.join(imagerydir,test),os.path.join(testdir,test))
    train_list = list(set(next(os.walk(imagerydir))[2]) - set(test_list))
    for train in train_list:
        shutil.copyfile(os.path.join(imagerydir,train),os.path.join(traindir,train))
    print(len(train_list))
    print(len(test_list))
    
train_test_split(EIGHTCHANNELDIR,TRAIN_DIR, TEST_DIR, .1)

groundtruth_list = next(os.walk(GROUNDTRUTH_DIR))[2]
for file in groundtruth_list:
    shutil.copyfile(os.path.join(GROUNDTRUTH_DIR,file),os.path.join(VALIDATION_DIR,file))

461
51


In [2]:
class ImageryConfig(Config):
    """Configuration for training on worldview-2 imagery. 
    Will eventually want to make this a sub-class of a 
    larger Imagery class. Overrides values specific to WV2.
    
    Descriptive documentation for each attribute is at
    https://github.com/matterport/Mask_RCNN/blob/master/mrcnn/config.py"""
    
    def __init__(self, N):
        """Set values of computed attributes. Channel dimension is overriden, 
        replaced 3 with N as per this guideline: https://github.com/matterport/Mask_RCNN/issues/314
        THERE MAY BE OTHER CODE CHANGES TO ACCOUNT FOR 3 vs N channels. See other 
        comments."""
        
        # Effective batch size
        self.BATCH_SIZE = self.IMAGES_PER_GPU * self.GPU_COUNT

        # Input image size
        if self.IMAGE_RESIZE_MODE == "crop":
            self.IMAGE_SHAPE = np.array([self.IMAGE_MIN_DIM, self.IMAGE_MIN_DIM, N])
        else:
            self.IMAGE_SHAPE = np.array([self.IMAGE_MAX_DIM, self.IMAGE_MAX_DIM, N])

        # Image meta data length
        # See compose_image_meta() for details
        self.IMAGE_META_SIZE = 1 + 3 + 3 + 4 + 1 + self.NUM_CLASSES

    
    # Give the configuration a recognizable name
    NAME = "wv2-subsets"

    # Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 2
    IMAGES_PER_GPU = 4

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + ag

    # Use small images for faster training. Determines the image shape.
    # From build() in model.py
    # Exception("Image size must be dividable by 2 at least 6 times "
                       #     "to avoid fractions when downscaling and upscaling."
                       #    "For example, use 256, 320, 384, 448, 512, ... etc. "
    IMAGE_MIN_DIM = 300
    IMAGE_MAX_DIM = 300

    # Use smaller anchors because our image and objects are small.
    # Setting Large upper scale since some fields take up nearly 
    # whole image
    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 300)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 1000

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 50

In [3]:
class ImageryDataset(utils.Dataset):
    """Generates the Imagery dataset."""
    
    def load_image(self, image_id):
        """Load the specified image and return a [H,W,4] Numpy array.
        Channels are ordered [B, G, R, NIR]. This is called by the 
        Keras data_generator function
        """
        # Load image
        image = skimage.io.imread(self.image_info[image_id]['path'])
    
        assert image.shape[-1] == 8
        assert image.ndim == 3
    
        return image
    
    def load_wv2(self, dataset_dir, subset):
        """Load a subset of the nuclei dataset.

        dataset_dir: Root directory of the dataset
        subset: Subset to load.
                * train: stage1_train excluding validation images
                * val: validation images from VAL_IMAGE_IDS
        """
        # Add classes. We have one class.
        # Naming the dataset wv2, and the class agriculture
        self.add_class("wv2", 1, "agriculture")

        # Which subset?
        # "test": use TESTDIR
        # "train": use TRAINDIR
        assert subset in ["train", "test"]
        image_dir = os.path.join(dataset_dir, subset)
    
        # Get image ids from directory names
        image_paths = next(os.walk(image_dir))[2]

        # Add images
        for image_path in image_paths:
            self.add_image(
                "wv2",
                image_id=image_path[-20:-12],
                path=image_path)
    
    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        info = self.image_info[image_id]
        # Get mask directory from image path
        mask_dir = VALIDATION_DIR

        # Read mask files from .png image
        mask = []
        for f in next(os.walk(mask_dir))[2]:
            if f.endswith(".png"):
                m = skimage.io.imread(os.path.join(mask_dir, f)).astype(np.bool)
                mask.append(m)
        mask = np.stack(mask, axis=-1)
        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID, we return an array of ones
        return mask, np.ones([mask.shape[-1]], dtype=np.int32)
    
    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "nucleus":
            return info["id"]
        else:
            super(self.__class__, self).image_reference(image_id)

SyntaxError: positional argument follows keyword argument (<ipython-input-3-2ddd4ca33436>, line 35)

In [4]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
dataset_train = ImageryDataset()
dataset_train.add_image()
dataset_train.load_image()
# dataset_train.prepare()